In [1]:
"""Script:  Automatically counts alleles in GT_variation data after filtering"""

#  Used modules

import pandas as pd
import numpy as np

In [2]:
# Loading main files

##  Exones data
exones = pd.read_excel("Exones_variance_pdfilter.xlsx")

##  Genes data
genes  = pd.read_excel("Genes_variance_pdfilter.xlsx")

In [3]:
#  Used columns - same for exones & genes

CS_columns = [i for i in range(1,64)]
CN_columns = [i for i in range(64,97) if i != 85] ## 85 skipped - duplicated in data

#  Summary columns name

## named in order - 1/1, 0/1, 0/0, 0/n (n - positive integer), n/n
## numbers - related to alleles in vcf

columns_case = ["CS_AA", "CS_AR", "CS_RR", "CS_RO", "CS_OO"]
columns_control = ["CN_AA","CN_AR","CN_RR", "CN_RO", "CN_OO"]

#### Alleles counting exone data

In [4]:
# Copy main dataset - against possible fragmentations

exones_copy = exones.copy()



#  Division data by : → extraction only GT for sample

## CS

for i in CS_columns:
    exones_copy[f"GT_{i}"] = exones_copy[i].astype(str).str.split(":").str[0]

## CN
    
for i in CN_columns:
    exones_copy[f"GT_{i}"] = exones_copy[i].astype(str).str.split(":").str[0]

In [5]:
#  creating aummary columns

## CS

exones_copy["CS_AA"] = 0
exones_copy["CS_AR"] = 0
exones_copy["CS_RR"] = 0
exones_copy["CS_RO"] = 0
exones_copy["CS_OO"] = 0


## CN

exones_copy["CN_AA"] = 0
exones_copy["CN_AR"] = 0
exones_copy["CN_RR"] = 0
exones_copy["CN_RO"] = 0
exones_copy["CN_OO"] = 0

# adding specific genotypes
    
## CS
    
for i in CS_columns:
    ###  Counts 1/1, 0/1, 0/1
    exones_copy["CS_AA"] += exones_copy[f"GT_{i}"].astype(str).str.count(r'^1/1').astype(int)
    exones_copy["CS_AR"] += exones_copy[f"GT_{i}"].astype(str).str.count(r'^0/1').astype(int)
    exones_copy["CS_RR"] += exones_copy[f"GT_{i}"].astype(str).str.count(r'^0/0').astype(int)

    ###  Counts 0/n, n/n - n = positive integer 1 to 19
    exones_copy["CS_RO"] += exones_copy[f"GT_{i}"].astype(str).str.count(r'^0/[1-9]|1[0-9]').astype(int)
    exones_copy["CS_OO"] += exones_copy[f"GT_{i}"].astype(str).str.count(r'^[1-9]|1[0-9]/[1-9]|1[0-9]').astype(int)

## CN

for i in CN_columns:
    ###  Counts 1/1, 0/1, 0/1
    exones_copy["CN_AA"] += exones_copy[f"GT_{i}"].astype(str).str.count(r'^1/1').astype(int)
    exones_copy["CN_AR"] += exones_copy[f"GT_{i}"].astype(str).str.count(r'^0/1').astype(int)
    exones_copy["CN_RR"] += exones_copy[f"GT_{i}"].astype(str).str.count(r'^0/0').astype(int)

    ###  Counts 0/n, n/n - n = positive integer 1 to 19
    exones_copy["CN_RO"] += exones_copy[f"GT_{i}"].astype(str).str.count(r'^0/[1-9]|1[0-9]').astype(int)
    exones_copy["CN_OO"] += exones_copy[f"GT_{i}"].astype(str).str.count(r'^[1-9]|1[0-9]/[1-9]|1[0-9]').astype(int)

#  Creating alleles columns
    
##  aleles 0 and 1

exones_copy["CS_A"] = exones_copy["CS_AA"] * 2 + exones_copy["CS_AR"]
exones_copy["CS_R"] = exones_copy["CS_RR"] * 2 + exones_copy["CS_AR"]
exones_copy["CN_A"] = exones_copy["CN_AA"] * 2 + exones_copy["CN_AR"]
exones_copy["CN_R"] = exones_copy["CN_RR"] * 2 + exones_copy["CN_AR"]

##  aleles 0 and others

exones_copy["CS_R"] = exones_copy["CS_RR"] * 2 + exones_copy["CS_RO"]
exones_copy["CS_O"] = exones_copy["CS_OO"] * 2 + exones_copy["CS_RO"]
exones_copy["CN_R"] = exones_copy["CN_RR"] * 2 + exones_copy["CN_RO"]
exones_copy["CN_O"] = exones_copy["CN_OO"] * 2 + exones_copy["CN_RO"]

#  Checking if sum of columns equal 95
    
##  1 == total is 95
##  0 == total not 95
    
exones_copy["Total"] = np.where(exones_copy["CS_AA"] + exones_copy["CS_AR"] + exones_copy["CS_RR"] + exones_copy["CN_AA"] + exones_copy["CN_AR"] + exones_copy["CN_RR"] == 95, 1, 0)


#  Creating two datasets - multi (multi alt), singl (only one alt)

exones_multi = exones_copy.loc[exones_copy["Total"] == 0]
exones_single = exones_copy.loc[exones_copy["Total"] == 1]

C:\Users\chumi\AppData\Local\Temp\ipykernel_12080\593134813.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  exones_copy["CS_RR"] = 0
C:\Users\chumi\AppData\Local\Temp\ipykernel_12080\593134813.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  exones_copy["CS_RO"] = 0
C:\Users\chumi\AppData\Local\Temp\ipykernel_12080\593134813.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.c

In [6]:
#  Export final files

## Multi

exones_multi.to_excel("Exones_variance_pdfilter_allelesdone_multi.xlsx", index=False)

## Single

exones_single.to_excel("Exones_varace_pdfilter_allelesdone_single.xlsx", index=False)

#### Alleles counting genes data


In [7]:
#  Copy main dataset agains fragmentation

genes_copy = genes.copy()


#  Division data by : → extraction only GT for sample

## CS

for i in CS_columns:
    genes_copy[f"GT_{i}"] = genes_copy[i].astype(str).str.split(":").str[0]

## CN
    
for i in CN_columns:
    genes_copy[f"GT_{i}"] = genes_copy[i].astype(str).str.split(":").str[0]

In [8]:
#  creating aummary columns

## CS

genes_copy["CS_AA"] = 0
genes_copy["CS_AR"] = 0
genes_copy["CS_RR"] = 0
genes_copy["CS_RO"] = 0
genes_copy["CS_OO"] = 0


## CN

genes_copy["CN_AA"] = 0
genes_copy["CN_AR"] = 0
genes_copy["CN_RR"] = 0
genes_copy["CN_RO"] = 0
genes_copy["CN_OO"] = 0

# adding specific genotypes
    
## CS
    
for i in CS_columns:
    ###  Counts 1/1, 0/1, 0/1
    genes_copy["CS_AA"] += genes_copy[f"GT_{i}"].astype(str).str.count(r'^1/1').astype(int)
    genes_copy["CS_AR"] += genes_copy[f"GT_{i}"].astype(str).str.count(r'^0/1').astype(int)
    genes_copy["CS_RR"] += genes_copy[f"GT_{i}"].astype(str).str.count(r'^0/0').astype(int)

    ###  Counts 0/n, n/n - n = positive integer 1 to 19
    genes_copy["CS_RO"] += genes_copy[f"GT_{i}"].astype(str).str.count(r'^0/[1-9]|1[0-9]').astype(int)
    genes_copy["CS_OO"] += genes_copy[f"GT_{i}"].astype(str).str.count(r'^[1-9]|1[0-9]/[1-9]|1[0-9]').astype(int)

## CN

for i in CN_columns:
    ###  Counts 1/1, 0/1, 0/1
    genes_copy["CN_AA"] += genes_copy[f"GT_{i}"].astype(str).str.count(r'^1/1').astype(int)
    genes_copy["CN_AR"] += genes_copy[f"GT_{i}"].astype(str).str.count(r'^0/1').astype(int)
    genes_copy["CN_RR"] += genes_copy[f"GT_{i}"].astype(str).str.count(r'^0/0').astype(int)

    ###  Counts 0/n, n/n - n = positive integer 1 to 19
    genes_copy["CN_RO"] += genes_copy[f"GT_{i}"].astype(str).str.count(r'^0/[1-9]|1[0-9]').astype(int)
    genes_copy["CN_OO"] += genes_copy[f"GT_{i}"].astype(str).str.count(r'^[1-9]|1[0-9]/[1-9]|1[0-9]').astype(int)

#  Creating alleles columns

##  aleles 0 and 1

genes_copy["CS_A"] = genes_copy["CS_AA"] * 2 + genes_copy["CS_AR"]
genes_copy["CS_R"] = genes_copy["CS_RR"] * 2 + genes_copy["CS_AR"]
genes_copy["CN_A"] = genes_copy["CN_AA"] * 2 + genes_copy["CN_AR"]
genes_copy["CN_R"] = genes_copy["CN_RR"] * 2 + genes_copy["CN_AR"]

##  aleles 0 and others

genes_copy["CS_R"] = genes_copy["CS_RR"] * 2 + genes_copy["CS_RO"]
genes_copy["CS_O"] = genes_copy["CS_OO"] * 2 + genes_copy["CS_RO"]
genes_copy["CN_R"] = genes_copy["CN_RR"] * 2 + genes_copy["CN_RO"]
genes_copy["CN_O"] = genes_copy["CN_OO"] * 2 + genes_copy["CN_RO"]


#  Checking if sum of columns equal 95
    
##  1 == total is 95
##  0 == total not 95
    
genes_copy["Total"] = np.where(genes_copy["CS_AA"] + genes_copy["CS_AR"] + genes_copy["CS_RR"] + genes_copy["CN_AA"] + genes_copy["CN_AR"] + genes_copy["CN_RR"] == 95, 1, 0)


#  Creating two datasets - multi (multi alt), singl (only one alt)

genes_multi = genes_copy.loc[genes_copy["Total"] == 0]
genes_single = genes_copy.loc[genes_copy["Total"] == 1]

C:\Users\chumi\AppData\Local\Temp\ipykernel_12080\3602881067.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  genes_copy["CS_RR"] = 0
C:\Users\chumi\AppData\Local\Temp\ipykernel_12080\3602881067.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  genes_copy["CS_RO"] = 0
C:\Users\chumi\AppData\Local\Temp\ipykernel_12080\3602881067.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.

In [9]:
#  Export final files

## Multi

genes_multi.to_excel("Genes_variance_pdfilter_allelesdone_multi.xlsx", index=False)

## Single

genes_single.to_excel("Genes_varace_pdfilter_allelesdone_single.xlsx", index=False)